In [1]:
%pip install -Uq "unstructured[all-docs]" pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from unstructured.partition.pdf import partition_pdf
import os

path = os.path.join(os.getcwd(), "RRMenu.pdf")

chunks = partition_pdf(
    filename=path,                              # only required parameter
    infer_table_structure=False,                # extract tables
    strategy="auto",                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    skip_infer_table_types=True,
    # extract_image_block_types=["Image"],      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",   # if none, images and tables will be saved in base64


    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",               # or "basic" | Menu has clear titles
    max_characters=10000,                       # max size of chunk
    combine_text_under_n_chars=2000,            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                      # start new part after 6000 characters
    )

/Users/alekmelenski/Desktop/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(len(chunks))

9


In [5]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '43c2099fd4e91cc48e55834faccbbcbd',
 'text': 'GOURMET FIRE-GRILLED BURGERS\n\nALL BURGERS INCLUDE A BOTTOMLESS SIDE\n\nSteak Fries • Yukon Chips • Steamed Broccoli • Side Salad Sweet Potato Fries (1.79) • Garlic Fries (1.79)\n\nOUR SIGNATURES Burgers you won’t find anywhere else. 100% Fresh, Never Frozen Beef.\n\nTHE SOUTHERN CHARM BURGER® Brown sugar glaze, candied bacon, Whiskey River® BBQ Sauce, Cheddar, caramelized onions, lettuce and mayo on a toasted brioche bun. 14.99 cal 1190\n\nBLACK & BLEU\n\nSautéed and blackened portobello mushrooms, caramelized onions, creamy cheese sauce, Bleu cheese crumbles, lettuce and roasted garlic aioli on a toasted brioche bun. 15.39 cal 980\n\nWHISKEY RIVER® BBQ\n\nDIP IT. DUNK IT. DOUSE YOUR BURGER.\n\nWhiskey River® BBQ Sauce, crispy onion straws, Cheddar, lettuce, tomatoes and mayo on a toasted sesame bun. 13.79 cal 1140 THE MADLOVE BURGER A Cheddar-and-Parmesan crisp, Provolone, Swiss, jalapeño relis

In [6]:
texts = []

for chunk in chunks:
    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# Constants for folders
MODEL_DIR = os.path.join(os.getcwd(), "model_dir")
TOKENIZER_DIR = os.path.join(MODEL_DIR, "tokenizer")
OFFLOAD_FOLDER = os.path.join(MODEL_DIR, "offload")
MODEL_WEIGHTS_DIR = os.path.join(MODEL_DIR, "model")  # model weights

# Ensure directories exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TOKENIZER_DIR, exist_ok=True)
os.makedirs(OFFLOAD_FOLDER, exist_ok=True)
os.makedirs(MODEL_WEIGHTS_DIR, exist_ok=True)

MODELNAME = "mistralai/Mistral-7B-Instruct-v0.1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODELNAME,
    cache_dir=TOKENIZER_DIR
)

# Load model with offloading
MODEL = AutoModelForCausalLM.from_pretrained(
    MODELNAME,
    device_map="auto",              # tries gpu, then cpu
    offload_folder=OFFLOAD_FOLDER,
    cache_dir=MODEL_WEIGHTS_DIR,
    dtype=torch.float16             # half the size of weight
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.12s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
#Summaries
prompt_template = """
You are an assistant tasked with summarizing text.
Give a concise summary of the text.

Respond only with the summary, no additional comments.
Do not start your message by saying "Here is a summary" or anything like that.
Just give me the summary as it is.

Text chunk: {element}
"""

def summarize_text(element, max_new_tokens=200):
    prompt = prompt_template.format(element=element)
    inputs = tokenizer(prompt, return_tensors="pt").to(MODEL.device)
    outputs = MODEL.generate(**inputs,
                            max_new_tokens=max_new_tokens,
                            temperature=0.7,                      # more likely to select most probable word (logit / 0.7)
                            do_sample=True,                       # need to set this to true for temperature to have any effect
                            top_p=0.9,                            # model choses from top tokens that make up 90% of the probability mass (ignores super low prob tokens from being selected)
                            eos_token_id=tokenizer.eos_token_id
                            )                            
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the original prompt to get only the model’s output
    summary = generated_text[len(prompt):].strip()
    return summary

summary = []
for chunk in texts:
    summary.append(summarize_text(chunk))
    print("Summary Generated")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary Generated


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary Generated


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Summary Generated


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary Generated


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary Generated
Summary Generated


In [ ]:

print(summary[3])

You are an assistant tasked with summarizing text.
Give a concise summary of the text.

Respond only with the summary, no additional comments.
Do not start your message by saying "Here is a summary" or anything like that.
Just give me the summary as it is.

Text chunk: FRESHLY BAKED PIZZA Toppings Loaded Edge to Edge®.

TERIYAKI CHICKEN

Teriyaki, grilled pineapple, Swiss, lettuce, tomatoes and mayo on a toasted sesame bun. 13.99 cal 780

SIGNATURE PIZZAS

FAMOUS THIN Small 10" 12.99 100–150 cal/slice* Large 14" 17.99 110–170 cal/slice*

CAULIFLOWER Small 10" 14.99 70–130 cal/slice*

GLUTEN FREE†† 12" 17.99 110–150 cal/slice*

WHISKEY RIVER® BBQ CHICKEN

New and improved recipe! Sliced chicken breast, smoked Provolone, shredded Cheddar and red onions, drizzled with Whiskey River® BBQ sauce. Not available in Gluten Free 12".

CALIFORNIA CHICKEN House-made guac, hardwood-smoked bacon, Provolone, pickles, lettuce, tomatoes and mayo on a toasted sesame bun. 14.69 cal 720

WHISKEY RIVER® BB